In [234]:
import json
import numpy as np
import pandas as pd

In [235]:
with open('test.json') as file:
    data = json.load(file)

match data['goal']:
    case 'min':
        f = -1 * np.array(data['f'])
    case 'max':
        f = np.array(data['f'])

n = len(data['constraints'][0]['coefs'])
m = len(data['constraints'])
A = np.zeros((0, n))
b = np.empty((0, 1))
for constraint in data['constraints']:
    coefs = np.array(constraint['coefs'])
    match constraint['type']:
        case 'eq':
            if len(coefs) != A.shape[1]:
                coefs = np.pad(coefs, (0, A.shape[1] - len(coefs)), 'constant', constant_values=0)
            A = np.vstack([A, coefs])
            b = np.append(b, constraint['b'])
        case 'lte':
            if len(coefs) != A.shape[1]:
                coefs = np.pad(coefs, (0, A.shape[1] - len(coefs)), 'constant', constant_values=0)
            coefs = np.append(coefs, 1)
            A = np.hstack([A, np.zeros((A.shape[0], 1))])
            A = np.vstack([A, coefs])
            b = np.append(b, constraint['b'])
        case 'gte':
            if len(coefs) != A.shape[1]:
                coefs = np.pad(coefs, (0, A.shape[1] - len(coefs)), 'constant', constant_values=0)
            coefs = np.append(coefs, -1)
            A = np.hstack([A, np.zeros((A.shape[0], 1))])
            A = np.vstack([A, coefs])
            b = np.append(b, constraint['b'])
f = np.pad(f, (0, A.shape[1] - len(f)), 'constant', constant_values=0) if A.shape[1] > len(f) else f
f, A, b

(array([ 0,  0,  0, -1,  1]),
 array([[ 1.,  0.,  0.,  1., -2.],
        [ 0.,  1.,  0., -2.,  1.],
        [ 0.,  0.,  1.,  3.,  1.]]),
 array([1., 2., 3.]))

In [236]:
A_ = np.array([[-1, 1, 1, 2, 1],
             [1, 0, 0, 1, -5],
             [0, 1, 0, 5, 1]])

b_ = np.array([2, 4, 4])

c_ = np.array([1, 1, 0, 0, 0])

In [237]:
def to_tableau(c, A, b):
    xb = np.hstack((A, b.reshape(b.shape[0], 1)))
    z = np.append(c, [0])
    return np.vstack((xb, z))

def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])

import math

def get_pivot_position(tableau):
    z = tableau[-1]
    #column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    column = np.argmax(z[:-1])

    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)
    
    if (all([r == math.inf for r in restrictions])):
        raise Exception("Linear program is unbounded.")

    row = restrictions.index(min(restrictions))
    return row, column

def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]

    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value

    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier

    return new_tableau

def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns[:-1]:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)

    return solutions

def simplex(c, A, b):
    tableau = to_tableau(c, A, b)
    print(pd.DataFrame(tableau), '\n')
    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)
        print(pd.DataFrame(tableau), '\n')
    return get_solution(tableau)

def simplex_inner(tableau):
    print(pd.DataFrame(tableau), '\n')
    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)
        print(pd.DataFrame(tableau), '\n')
    return tableau

In [238]:
simplex(f, A, b)

     0    1    2    3    4    5
0  1.0  0.0  0.0  1.0 -2.0  1.0
1  0.0  1.0  0.0 -2.0  1.0  2.0
2  0.0  0.0  1.0  3.0  1.0  3.0
3  0.0  0.0  0.0 -1.0  1.0  0.0 

     0    1    2    3    4    5
0  1.0  2.0  0.0 -3.0  0.0  5.0
1  0.0  1.0  0.0 -2.0  1.0  2.0
2  0.0 -1.0  1.0  5.0  0.0  1.0
3  0.0 -1.0  0.0  1.0  0.0 -2.0 

     0    1    2    3    4    5
0  1.0  1.4  0.6  0.0  0.0  5.6
1  0.0  0.6  0.4  0.0  1.0  2.4
2  0.0 -0.2  0.2  1.0  0.0  0.2
3  0.0 -0.8 -0.2  0.0  0.0 -2.2 



[5.6, 0, 0, 0.2, 2.4]

In [239]:
simplex(c_, A_, b_)

   0  1  2  3  4  5
0 -1  1  1  2  1  2
1  1  0  0  1 -5  4
2  0  1  0  5  1  4
3  1  1  0  0  0  0 

     0    1    2    3    4    5
0  0.0  1.0  1.0  3.0 -4.0  6.0
1  1.0  0.0  0.0  1.0 -5.0  4.0
2  0.0  1.0  0.0  5.0  1.0  4.0
3  0.0  1.0  0.0 -1.0  5.0 -4.0 

     0    1    2     3    4     5
0  0.0  5.0  1.0  23.0  0.0  22.0
1  1.0  5.0  0.0  26.0  0.0  24.0
2  0.0  1.0  0.0   5.0  1.0   4.0
3  0.0 -4.0  0.0 -26.0  0.0 -24.0 



[24.0, 0, 22.0, 0, 4.0]

In [240]:
def count_basic(columns):
    i = 0
    for col in columns:
        if is_basic(col):
            i += 1
    return i

In [241]:
tableau = to_tableau(f, A, b)
cols = np.array(tableau)[:-1].T[:-1]
if (cols.shape[1] != count_basic(cols)):
    print(1)

In [242]:
tableau = to_tableau(c_, A_, b_)
cols = np.array(tableau)[:-1].T[:-1]
if (cols.shape[1] != count_basic(cols)):
    print(tableau)

[[-1  1  1  2  1  2]
 [ 1  0  0  1 -5  4]
 [ 0  1  0  5  1  4]
 [ 1  1  0  0  0  0]]


In [243]:
def get_nonbasic_rows(tableau):
    cols = np.array(tableau)[:-1].T[:-1]
    res = set()
    for col in cols:
        if is_basic(col):
            res.update(np.nonzero(col)[0])
    return list(set(range(cols.shape[1])) ^ res)

In [244]:
def get_row(tableau, col_index):
    col = tableau[:, col_index]
    if is_basic(col):
        res = tableau[np.nonzero(col)[0]].ravel()
        res[col_index] = 0
        return -1 * res
    res = np.zeros(shape=tableau.shape[1])
    res[col_index] = 1
    return res

In [245]:
def find_start_plane(tableau):
    func = tableau[-1, :-1].copy()
    bad_rows = get_nonbasic_rows(tableau)
    tableau[-1] = np.sum([tableau[i] for i in bad_rows], axis=0)
    bad_cols = []
    for i in bad_rows:
        new_var = np.zeros(shape=(tableau.shape[0], 1))
        new_var[i] = 1
        bad_cols.append(tableau.shape[1] - 1)
        tableau = np.hstack((tableau[:, :-1], new_var, tableau[:, -1:]))
    
    t = np.round(simplex_inner(tableau), 6)
    t = np.delete(t, bad_cols, axis=1)
    print(get_solution(t))
    print(func)
    new_func = np.zeros(shape=t.shape[1])
    for i in np.nonzero(func)[0]:
        new_func += func[i] * get_row(t, i)
    t[-1] = new_func
    return t

In [246]:
tableau = to_tableau(c_, A_, b_)
t_test = find_start_plane(tableau)
t_test

     0    1    2    3    4    5    6    7
0 -1.0  1.0  1.0  2.0  1.0  0.0  0.0  2.0
1  1.0  0.0  0.0  1.0 -5.0  1.0  0.0  4.0
2  0.0  1.0  0.0  5.0  1.0  0.0  1.0  4.0
3  1.0  1.0  0.0  6.0 -4.0  0.0  0.0  8.0 

     0    1    2    3    4    5    6    7
0 -1.0  0.6  1.0  0.0  0.6  0.0 -0.4  0.4
1  1.0 -0.2  0.0  0.0 -5.2  1.0 -0.2  3.2
2  0.0  0.2  0.0  1.0  0.2  0.0  0.2  0.8
3  1.0 -0.2  0.0  0.0 -5.2  0.0 -1.2  3.2 

     0             1    2    3    4    5    6             7
0  0.0  4.000000e-01  1.0  0.0 -4.6  1.0 -0.6  3.600000e+00
1  1.0 -2.000000e-01  0.0  0.0 -5.2  1.0 -0.2  3.200000e+00
2  0.0  2.000000e-01  0.0  1.0  0.2  0.0  0.2  8.000000e-01
3  0.0 -1.665335e-16  0.0  0.0  0.0 -1.0 -1.0 -8.881784e-16 

[3.2, 0, 3.6, 0.8, 0]
[1 1 0 0 0]


array([[ 0. ,  0.4,  1. ,  0. , -4.6,  3.6],
       [ 1. , -0.2,  0. ,  0. , -5.2,  3.2],
       [ 0. ,  0.2,  0. ,  1. ,  0.2,  0.8],
       [ 0. ,  1.2,  0. ,  0. ,  5.2, -3.2]])

In [247]:
simplex_inner(t_test)

     0    1    2    3    4    5
0  0.0  0.4  1.0  0.0 -4.6  3.6
1  1.0 -0.2  0.0  0.0 -5.2  3.2
2  0.0  0.2  0.0  1.0  0.2  0.8
3  0.0  1.2  0.0  0.0  5.2 -3.2 

     0    1    2     3    4     5
0  0.0  5.0  1.0  23.0  0.0  22.0
1  1.0  5.0  0.0  26.0  0.0  24.0
2  0.0  1.0  0.0   5.0  1.0   4.0
3  0.0 -4.0  0.0 -26.0  0.0 -24.0 



[array([ 0.,  5.,  1., 23.,  0., 22.]),
 array([ 1.,  5.,  0., 26.,  0., 24.]),
 array([0., 1., 0., 5., 1., 4.]),
 array([  0.,  -4.,   0., -26.,   0., -24.])]